# Exp R1a: Hysteresis Loop (η=0.4, Fast Sweep)

## 目的
真のヒステリシス・ループを測定：λを段階的に上昇→下降させ、m(λ)の上り枝／下り枝を記録。

## 実験設計
- **Sweep**: λ = 0.20 → 0.65 → 0.20 (stepwise)
- **Step size**: Δλ = 0.02 (boundary近傍 0.42-0.58 は Δλ = 0.01)
- **Hold time**: Fast (1 epoch per λ step)
- **η**: 0.4
- **Seeds**: 0-9 (10 runs)
- **Total**: 10 runs

## 推定時間
~10 runs × 45 min ≈ **7.5 時間**

## 関連ノートブック
- R1a (this): η=0.4, fast
- R1b: η=0.4, slow
- R2a: η=0.8, fast
- R2b: η=0.8, slow

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_R1_hysteresis_loop'
NOTEBOOK_ID = 'R1a'  # このノートブックの識別子
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

# 既存ディレクトリを検索（R1a/b で共有）
existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Save directory: {SAVE_DIR}')
print(f'This notebook: {NOTEBOOK_ID} (η=0.4, fast)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== ResNet18 for CIFAR-10 =====
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

# ===== IndexedDataset =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
# ===== λ Sweep Schedule for Hysteresis Loop =====

def generate_lambda_sweep(eta):
    """
    Generate λ values for hysteresis loop: up sweep + down sweep
    Higher resolution near the transition boundary
    """
    if eta == 0.4:
        boundary_low, boundary_high = 0.42, 0.58
    else:  # eta == 0.8
        boundary_low, boundary_high = 0.38, 0.54
    
    lambda_values = []
    
    # Up sweep: 0.20 → 0.65
    lam = 0.20
    while lam <= 0.65:
        lambda_values.append(('up', round(lam, 3)))
        # High resolution near boundary
        if boundary_low <= lam <= boundary_high:
            lam += 0.01
        else:
            lam += 0.02
    
    # Down sweep: 0.65 → 0.20 (excluding 0.65 to avoid duplicate)
    lam = 0.65 - 0.02
    while lam >= 0.20:
        lambda_values.append(('down', round(lam, 3)))
        if boundary_low <= lam <= boundary_high:
            lam -= 0.01
        else:
            lam -= 0.02
    
    return lambda_values

# Test
sweep = generate_lambda_sweep(0.4)
up_count = sum(1 for s in sweep if s[0] == 'up')
down_count = sum(1 for s in sweep if s[0] == 'down')
print(f'Total λ steps: {len(sweep)} (up: {up_count}, down: {down_count})')
print(f'First 10: {sweep[:10]}')
print(f'Last 10: {sweep[-10:]}')

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# このノートブック専用
NOISE_RATE = 0.4
HOLD_EPOCHS = 1  # Fast: 1 epoch per λ step
SWEEP_TYPE = 'fast'

SEEDS = list(range(10))

LAMBDA_SWEEP = generate_lambda_sweep(NOISE_RATE)
TOTAL_EPOCHS = len(LAMBDA_SWEEP) * HOLD_EPOCHS

experiments = [{'seed': s, 'noise_rate': NOISE_RATE, 'sweep_type': SWEEP_TYPE} for s in SEEDS]

print(f'Noise rate: {NOISE_RATE}')
print(f'Sweep type: {SWEEP_TYPE} (hold={HOLD_EPOCHS} epochs per λ)')
print(f'λ steps: {len(LAMBDA_SWEEP)}')
print(f'Total epochs per run: {TOTAL_EPOCHS}')
print(f'Seeds: {SEEDS}')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * TOTAL_EPOCHS * 0.5 / 60:.1f} hours')

# Save config
config = {
    'experiment': EXP_NAME,
    'notebook_id': NOTEBOOK_ID,
    'description': 'Hysteresis loop measurement with stepwise λ sweep',
    'parameters': {
        'noise_rate': NOISE_RATE,
        'sweep_type': SWEEP_TYPE,
        'hold_epochs': HOLD_EPOCHS,
        'lambda_steps': len(LAMBDA_SWEEP),
        'total_epochs': TOTAL_EPOCHS,
        'seeds': SEEDS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    }
}

with open(f'{SAVE_DIR}/{NOTEBOOK_ID}_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print(f'Config saved')

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy_labels = labels.copy()
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_indices = np.random.choice(n_samples, n_noisy, replace=False)
    for idx in noisy_indices:
        original = labels[idx]
        new_label = np.random.choice([l for l in range(10) if l != original])
        noisy_labels[idx] = new_label
    return noisy_labels

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Hysteresis Loop Training =====
def train_hysteresis_loop(model, train_loader, test_loader, clean_labels, noisy_labels, lambda_sweep, hold_epochs):
    """
    Train with stepwise λ sweep, recording test error at each λ step.
    """
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    global_epoch = 0
    
    # Results storage
    loop_data = []  # (direction, λ, test_error, test_acc)
    epoch_history = []  # Full epoch-by-epoch history
    
    total_steps = len(lambda_sweep)
    
    for step_idx, (direction, lam) in enumerate(lambda_sweep):
        # LR scheduling based on total epochs
        if global_epoch == 50:
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR * 0.1
        elif global_epoch == 75:
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR * 0.01
        
        # Train for hold_epochs at this λ
        for hold_ep in range(hold_epochs):
            model.train()
            epoch_cos = []
            
            for inputs, _, indices in train_loader:
                inputs = inputs.to(device, non_blocking=True)
                indices = indices.to(device, non_blocking=True)
                batch_noisy = noisy_labels_tensor[indices]
                batch_clean = clean_labels_tensor[indices]
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_struct = criterion(outputs, batch_noisy)
                loss_struct.backward(retain_graph=True)
                g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
                
                if global_step % K == 0 or cached_g_value is None:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss_value = criterion(outputs, batch_clean)
                    loss_value.backward()
                    cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
                
                g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
                g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
                
                cos_sim = (g_struct_norm @ g_value_norm).item()
                epoch_cos.append(cos_sim)
                
                g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
                
                optimizer.zero_grad()
                idx = 0
                for p in model.parameters():
                    numel = p.numel()
                    p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                    idx += numel
                optimizer.step()
                global_step += 1
            
            global_epoch += 1
        
        # Evaluate at end of this λ step
        test_acc = evaluate(model, test_loader)
        test_error = 1 - test_acc
        avg_cos = np.mean(epoch_cos) if epoch_cos else 0
        
        loop_data.append({
            'step_idx': step_idx,
            'direction': direction,
            'lambda': lam,
            'test_error': test_error,
            'test_acc': test_acc,
            'avg_cos': avg_cos,
            'global_epoch': global_epoch
        })
        
        if (step_idx + 1) % 10 == 0:
            print(f'    Step {step_idx+1}/{total_steps}: {direction} λ={lam:.3f} → error={test_error:.4f}')
    
    return loop_data

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = get_resnet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['noise_rate'], r['sweep_type'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    seed = exp['seed']
    eta = exp['noise_rate']
    stype = exp['sweep_type']
    
    if (eta, stype, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] η={eta} sweep={stype} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, eta, seed)
    
    model = get_resnet18().to(device)
    
    start_time = time.time()
    loop_data = train_hysteresis_loop(
        model, train_loader, test_loader, 
        clean_labels, noisy_labels, 
        LAMBDA_SWEEP, HOLD_EPOCHS
    )
    elapsed = time.time() - start_time
    
    # Extract up/down branches
    up_branch = [d for d in loop_data if d['direction'] == 'up']
    down_branch = [d for d in loop_data if d['direction'] == 'down']
    
    result = {
        'experiment_id': f'{NOTEBOOK_ID}-{run_num:03d}',
        'experiment': EXP_NAME,
        'notebook_id': NOTEBOOK_ID,
        'noise_rate': eta,
        'sweep_type': stype,
        'hold_epochs': HOLD_EPOCHS,
        'seed': seed,
        'time_seconds': elapsed,
        'loop_data': loop_data,
        'up_branch': up_branch,
        'down_branch': down_branch
    }
    results.append(result)
    completed.add((eta, stype, seed))
    
    # Summary
    up_errors = [d['test_error'] for d in up_branch]
    down_errors = [d['test_error'] for d in down_branch]
    print(f'  Up branch: min={min(up_errors):.4f}, max={max(up_errors):.4f}')
    print(f'  Down branch: min={min(down_errors):.4f}, max={max(down_errors):.4f}')
    print(f'  Time: {elapsed/60:.1f} min')
    
    # Save checkpoint
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta_hours = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta_hours:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print(f'{NOTEBOOK_ID} COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

# Flatten for CSV
rows = []
for r in results:
    for d in r['loop_data']:
        rows.append({
            'experiment_id': r['experiment_id'],
            'noise_rate': r['noise_rate'],
            'sweep_type': r['sweep_type'],
            'seed': r['seed'],
            'step_idx': d['step_idx'],
            'direction': d['direction'],
            'lambda': d['lambda'],
            'test_error': d['test_error'],
            'test_acc': d['test_acc'],
            'avg_cos': d['avg_cos']
        })

df = pd.DataFrame(rows)
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')
print(f'Total data points: {len(rows)}')

In [ ]:
# ===== ヒステリシス・ループ可視化 =====
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Individual loops
ax = axes[0]
for r in results:
    up_lam = [d['lambda'] for d in r['up_branch']]
    up_err = [d['test_error'] for d in r['up_branch']]
    down_lam = [d['lambda'] for d in r['down_branch']]
    down_err = [d['test_error'] for d in r['down_branch']]
    
    ax.plot(up_lam, up_err, 'b-', alpha=0.3, linewidth=0.8)
    ax.plot(down_lam, down_err, 'r-', alpha=0.3, linewidth=0.8)

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title(f'Hysteresis Loop (η={NOISE_RATE}, {SWEEP_TYPE})', fontsize=12)
ax.legend(['Up sweep', 'Down sweep'], loc='upper left')
ax.grid(True, alpha=0.3)

# Right: Mean loop with std
ax = axes[1]

# Aggregate by λ and direction
df_up = df[df['direction'] == 'up'].groupby('lambda')['test_error'].agg(['mean', 'std']).reset_index()
df_down = df[df['direction'] == 'down'].groupby('lambda')['test_error'].agg(['mean', 'std']).reset_index()

ax.errorbar(df_up['lambda'], df_up['mean'], yerr=df_up['std'], 
            fmt='b-o', markersize=3, capsize=2, label='Up sweep', alpha=0.8)
ax.errorbar(df_down['lambda'], df_down['mean'], yerr=df_down['std'], 
            fmt='r-o', markersize=3, capsize=2, label='Down sweep', alpha=0.8)

ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error (mean ± std)', fontsize=12)
ax.set_title(f'Mean Hysteresis Loop (η={NOISE_RATE}, {SWEEP_TYPE}, n={len(SEEDS)})', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_hysteresis_loop.png', dpi=150, bbox_inches='tight')
plt.show()

# Hysteresis area estimate
print('\n--- Hysteresis Analysis ---')
common_lambdas = set(df_up['lambda']) & set(df_down['lambda'])
if common_lambdas:
    area = 0
    sorted_lambdas = sorted(common_lambdas)
    for i in range(len(sorted_lambdas) - 1):
        lam1, lam2 = sorted_lambdas[i], sorted_lambdas[i+1]
        up1 = df_up[df_up['lambda'] == lam1]['mean'].values[0]
        up2 = df_up[df_up['lambda'] == lam2]['mean'].values[0]
        down1 = df_down[df_down['lambda'] == lam1]['mean'].values[0]
        down2 = df_down[df_down['lambda'] == lam2]['mean'].values[0]
        # Trapezoidal area
        area += abs((down1 - up1) + (down2 - up2)) / 2 * (lam2 - lam1)
    print(f'Estimated loop area: {area:.6f}')

print(f'\nNext: Run R1b (slow sweep) for comparison')